Importing required libraries and packages

In [0]:
!pip install 'nltk==3.4.5'

     |████████████████████████████████| 1.5MB 2.8MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449905 sha256=e3e9e79c6664159b5933acd6796794af23d7dbd4bca407cda3564fbdc226615c
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [0]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.utils import np_utils
from tensorflow.keras.callbacks import EarlyStopping
import sys

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Loading the IMDB dataset

In [0]:
path = "drive/My Drive/Colab Dataset/IMDB-Dataset.csv"
IMDBFull = pd.read_csv(path)

creating an object for the reviews separately

In [0]:
reviews=IMDBFull['review']

Extracting ordered vocabulary and creating mapping between the vocabulary and number index

In [0]:
vocab = sorted(list(set(reviews)))
char_to_index = dict((c, i) for i, c in enumerate(vocab))
index_to_char = dict((i, c) for i, c in enumerate(vocab))

In [0]:
len_review = len(reviews)
len_vocab = len(vocab)

Function to split the reviews with fixed length 50 and then using the previous indexing to convert character to integers.

In [0]:
len_sequence = 50
X_train = []
Y_train = []
for i in range(0, len_review - len_sequence, 1):
	sequence_input = reviews[i:i + len_sequence]
	sequence_output = reviews[i + len_sequence]
	X_train.append([char_to_index[char] for char in sequence_input])
	Y_train.append(char_to_index[sequence_output])

In [0]:
maxlen = len(X_train)

Re-scaling the patterns to be easily readable by LSTM

In [0]:
X = np.reshape(X_train, (maxlen, len_sequence, 1))
X = X/float(len_vocab)

In [0]:
Y = np_utils.to_categorical(Y_train)

Building a 3 LSTM layer sequential network with dropouts in each of them, introducing Early stopping to monitor the loss matrix

In [0]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(16, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(Y.shape[1], activation='softmax'))
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=2)

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
model.fit(X, Y, epochs=5, batch_size=512, callbacks=[es])

Epoch 1/5
49950/49950 [==============================] - 134s 3ms/step - loss: 10.8140
Epoch 2/5
49950/49950 [==============================] - 130s 3ms/step - loss: 10.8170
Epoch 3/5
49950/49950 [==============================] - 129s 3ms/step - loss: 10.8217
Epoch 00003: early stopping
